# Encryption as a Service: Transit Secrets Engine - Advanced

## Prerequisites

* Running Vault Server at localhost:8200 - see [Start Vault Server](./01-baseline/100-Setup-Vault.ipynb)

### Set Main Environment Variables

In [ ]:
export RED="\e[0;31m" YELLOW="\e[0;33m" BLDYELLOW="\e[1;33m" GREEN="\e[0;32m"
export CYAN="\e[0;36m" BLUE="\e[0;34m" WHITE="\e[0;37m" BLDWHITE="\e[1;37m"
export NC="\e[0m"
# Some commands may have sensitive information. Prevent commands starting with a space to be saved to shell history.
HISTCONTROL=ignoreboth # do not save lines that begin with space in history

printf "${GREEN}# Set MAIN_DIR for future reference.${NC}\n"
export MAIN_DIR=${PWD}

Be sure to replace `TF_ORG` and `TF_WS_NAME` with your own values.

In [ ]:
printf "${GREEN}# Set Vault Variables.${NC}\n"
export VAULT_PORT=8200
export VAULT_TOKEN=root
export VAULT_ADDR=http://localhost:${VAULT_PORT}
export VAULT_LICENSE=$(cat ../../license/vault.hclic)
# Set dir env vars. Create required directory.
WORK_DIR=config/vault
mkdir -p config/vault/{data,logs,config}
printf "\n$MAIN_DIR \n $WORK_DIR \n"

## Transit Engine for Sign and Verify

### Sign

This code shows the steps to enable the transit secret engine, configure a key, and use the sign leveraging Vault.

### Enable Transit engine

In [ ]:
vault secrets enable transit

### Create key
* Set `type` to `rsa-4096`.
* **NOTE**: Default key type (`aes256-gcm96`) doesn't support signing

CLI method

In [ ]:
vault write -f transit/keys/my-key type=rsa-4096

API method (illustration purposes only)

In [ ]:
curl -i \
    -H "X-Vault-Token: root" \
    -d '{"type":"rsa-4096"}' \
    -X POST "http://127.0.0.1:8200/v1/transit/keys/sign_key"

Read key information.

In [ ]:
vault read transit/keys/my-key

### Delete a key

In case you made mistakes and need to delete a key...

In [ ]:
#enable key deletion
vault write transit/keys/sign_key/config deletion_allowed=true
vault read transit/keys/sign_key #confirm deletion_allowed
vault delete transit/keys/sign_key # delete key
vault read transit/keys/sign_key #confirm key is gone

Encode a string as base64

In [ ]:
echo -n 'Created by Peter, you can trust me!' \
  | openssl base64 | tee $WORK_DIR/vault_b64.txt

Sample Output
```
Q3JlYXRlZCBieSBQZXRlciwgeW91IGNhbiB0cnVzdCBtZSE=
```

Sign the encoded data with your key through Vault.

In [ ]:
vault write \
    -field=signature \
    transit/sign/my-key \
    input=@$WORK_DIR/vault_b64.txt \
    > $WORK_DIR/vault_b64_signed.txt

cat $WORK_DIR/vault_b64_signed.txt

Sample Output
```
vault:v1:F+KxZxL6YX2fR7Wo65+jkpS3bTrrOfYdlXqxe01uAjle6zzEpi4n...
```

## Verify with the Key

Verify the signature of the file.

### Client with access to Vault

In [ ]:
# Verify on the receiving end
vault write transit/verify/my-key \
    input=@$WORK_DIR/vault_b64.txt \
    signature=@$WORK_DIR/vault_b64_signed.txt

You should see
```shell
Key      Value
---      -----
valid    true
```

### Command Summary for Client with access to Vault

In [ ]:
vault secrets enable transit
vault write -f transit/keys/my-key type=rsa-4096
#vault write -f transit/keys/bad-key type=rsa-4096

echo -n 'Created by Peter, you can trust me!' | base64 > $WORK_DIR/vault_b64.txt
cat $WORK_DIR/vault_b64.txt

#==> Sign the string
vault write \
    -field=signature \
    transit/sign/my-key \
    input=@$WORK_DIR/vault_b64.txt \
    > $WORK_DIR/vault_b64_signed.txt
cat $WORK_DIR/vault_b64_signed.txt

vault write \
    -field=signature \
    transit/sign/bad-key \
    input=@$WORK_DIR/vault_b64.txt \
    > $WORK_DIR/vault_b64_signed_bad.txt

#==> Verify the key: Client with access to Vault:
vault write transit/verify/my-key \
    input=@$WORK_DIR/vault_b64.txt \
    signature=@$WORK_DIR/vault_b64_signed.txt

vault write transit/verify/my-key \
    input=@$WORK_DIR/vault_b64.txt \
    signature=@$WORK_DIR/vault_b64_signed_bad.txt

### Offline client

#### Export Public Key

Export the **PUBLIC** key. Used for verification, so not sensitive.
* Grab content between (and including) "`-----BEGIN PUBLIC KEY-----`" and "`-----END PUBLIC KEY-----`"
* Put contents into a file: `public.key`
* If you have multiple versions of the key, you need to specify index. ex `.["1"].public_key`

In [ ]:
vault read -field=keys -format=json transit/keys/my-key | jq .[].public_key | cat

In [ ]:
vault read -field=keys -format=json transit/keys/my-key \
    | jq -r '.[].public_key' > $WORK_DIR/public.key
cat $WORK_DIR/public.key

Sample Output
```
-----BEGIN PUBLIC KEY-----
MIICIjANBgkqhkiG9w0BAQEFAAOCAg8AMIICCgKCAgEApvlAEqQwgyymVtMcVm5L
FuAKdU9XbL5CLC9Vi7V1COIgvgq57XV7GmcKyhb2/q6rJICsILwObytNzQEznjeE
FiTj2TfzobHAi70iRhP4vLtTdUnzNu6g+nx9iCPWN32AvMhk7VtHk7EX27d1ap4E
o8UCFokpzlTWF0trnfHhiMNcnxHILpedm3Rc6uSK/6zghMMXAT3TvPFZeQZMrilh
zprisdURUkfqVuUUk9DW+dMs1ABAUcKlAi/JbUBe1t8aDrkl7mEk58/tCndkrlyt
3tNBEoahcyCXgL/8lTNJwlw9mqQsTUv2zm/+WMrsxxNlTIu+Z40KGYdBDBXY3rNr
Eh2F7ATfpf5Ws6N5a8PkQ3o6ng2vME/PQZ4CXnnI1w/z+dFm6FlK/DOz+L7qCNI+
ewarl23eX1M/7WaZnYLlD63j1UWuO+aD+RzbsmGyl/5CC5MUnBhY9F+w5KA1fhz6
eRXELtYBHE8d0qV87CP+P9Es0XSWWuN1qE+qih3kNkAlWuHhlqGFxtZPOFSOokE8
J4LSehSU530VqxJzppkBrKnBj8AMAYMxvbUHsiyy4zmgdGYTX9/riccspws77Q+K
JwMaRWvVo7o0PcnfCEZTdAucKuvZZhfe8sbCeOqN+SlcRDXMXc/ChKCSZamSiFA7
4b2JEcdeuMCzMp8t0HZsD60CAwEAAQ==
-----END PUBLIC KEY-----
```

In [ ]:
echo "supersecret" > $WORK_DIR/in.txt.sha256

* Sign and base64 encode. Save output to `plaintext.sign` file.
	* When you sign, it's important to use the same `hash_algorithm` and `signature_algorithm` as OpenSSL.

In [ ]:
#--> Test Sign
cat $WORK_DIR/in.txt.sha256 | base64 \
    | vault write transit/sign/my-key input=-

In [ ]:
#==> Sign and encode.
cat $WORK_DIR/in.txt.sha256 | base64 \
    | vault write -format=json transit/sign/my-key input=- \
    | jq -r '.data.signature' \
    | sed 's|vault:v.:||g' | base64 -d \
    > $WORK_DIR/plaintext.sign

* We removed the vault:v1: prefix
  * `s|vault:v.:||g` - used "`.`" wildcard to match any version number.

Sample Output
```
Verified OK 
```

In [ ]:
openssl dgst -sha256 \
  -verify $WORK_DIR/public.key \
  -signature $WORK_DIR/plaintext.sign \
  -sigopt rsa_padding_mode:pss -sigopt rsa_pss_saltlen:-2 \
  $WORK_DIR/in.txt.sha256

In [ ]:
vault write -format=json transit/sign/my-key \
    input=@$WORK_DIR/vault_b64.txt \
    hash_algorithm=sha2-256 signature_algorithm=pkcs1v15 \
    | jq -r .data.signature > $WORK_DIR/msg.sig.base64

In [ ]:
cat $WORK_DIR/msg.sig.base64

* `hash_algorithm` - default is sha2-256
* `signature_algorithm` - default is pss

* Then create a file `msg.sig.base64` with the signature generated by Vault.
	* Vault always use base64 so the signature will be returned in base64.
	* OpenSSL only works with binary data, so you'll have to decode the signature first

In [ ]:
openssl base64 -d -A -in $WORK_DIR/msg.sig.base64 -out $WORK_DIR/msg.sig

And then verify the signature of your original message

In [ ]:
openssl dgst -sha256 -verify $WORK_DIR/public.key -signature $WORK_DIR/msg.sig msg.txt

### Rotate

```shell
export ENGINE_PATH=transit
export KEY_NAME=my-key
vault write -f ${ENGINE_PATH}/keys/${KEY_NAME}/rotate
```


Need to have OpenSSL use PSS padding:
```shell
openssl dgst -sha256 -verify public.key -signature msg.sig -sigopt rsa_padding_mode:pss msg.txt
```

## Resources
* Stenio - https://gist.github.com/stenio123/0ae467df32364efad0ca01d3b9c3e1c5
* https://gist.github.com/42wim/795acbdd90ac8a76fc78bc7f54ebbba7
* jwt - https://jwt.io/